# Assignment 1

In this assignment you will build a language model for the [OHHLA corpus](http://ohhla.com/) we are using in the book. You will train the model on the available training set, and can tune it on the development set. After submission we will run your notebook on a different test set. Your mark will depend on 

* whether your language model is **properly normalized**,
* its **perplexity** on the unseen test set,
* your **description** of your approach. 

To develop your model you have access to:

* The training and development data in `data/ohhla`.
* The code of the lecture, stored in a python module [here](/edit/statnlpbook/lm.py).
* Libraries on the [docker image](https://github.com/uclmr/stat-nlp-book/blob/python/Dockerfile) which contains everything in [this image](https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook), including scikit-learn and tensorflow. 

As we have to run the notebooks of all students, and because writing efficient code is important, **your notebook should run in 5 minutes at most**, on your machine. Further comments:

* We have tested a possible solution on the Azure VMs and it ran in seconds, so it is possible to train a reasonable LM on the data in reasonable time. 

* Try to run your parameter optimisation offline, such that in your answer notebook the best parameters are already set and don't need to be searched.

## Setup Instructions
It is important that this file is placed in the **correct directory**. It will not run otherwise. The correct directory is

    DIRECTORY_OF_YOUR_BOOK/assignments/2017/assignment1/problem/
    
where `DIRECTORY_OF_YOUR_BOOK` is a placeholder for the directory you downloaded the book to. After you placed it there, **rename the file** to your UCL ID (of the form `ucxxxxx`). 

## General Instructions
This notebook will be used by you to provide your solution, and by us to both assess your solution and enter your marks. It contains three types of sections:

1. **Setup** Sections: these sections set up code and resources for assessment. **Do not edit these**. 
2. **Assessment** Sections: these sections are used for both evaluating the output of your code, and for markers to enter their marks. **Do not edit these**. 
3. **Task** Sections: these sections require your solutions. They may contain stub code, and you are expected to edit this code. For free text answers simply edit the markdown field.  

Note that you are free to **create additional notebook cells** within a task section. 

Please **do not share** this assignment publicly, by uploading it online, emailing it to friends etc. 


## Submission Instructions

To submit your solution:

* Make sure that your solution is fully contained in this notebook. 
* **Rename this notebook to your UCL ID** (of the form "ucxxxxx"), if you have not already done so.
* Download the notebook in Jupyter via *File -> Download as -> Notebook (.ipynb)*.
* Upload the notebook to the Moodle submission site.


## <font color='green'>Setup 1</font>: Load Libraries
This cell loads libraries important for evaluation and assessment of your model. **Do not change it.**

In [2]:
#! SETUP 1
import sys, os
_snlp_book_dir = "../../../../"
sys.path.append(_snlp_book_dir) 
import statnlpbook.lm as lm
import statnlpbook.ohhla as ohhla
import math

## <font color='green'>Setup 2</font>: Load Training Data

This cell loads the training data. We use this data for assessment to define the reference vocabulary: the union of the words of the training and set set. You can use the dataset to train your model, but you are also free to load the data in a different way, or focus on subsets etc. However, when you do this, still **do not edit this setup section**. Instead refer to the variables in your own code, and slice and dice them as you see fit.   

In [3]:
#! SETUP 2
_snlp_train_dir = _snlp_book_dir + "/data/ohhla/train"
_snlp_dev_dir = _snlp_book_dir + "/data/ohhla/dev"
_snlp_train_song_words = ohhla.words(ohhla.load_all_songs(_snlp_train_dir))
_snlp_dev_song_words = ohhla.words(ohhla.load_all_songs(_snlp_dev_dir))
assert(len(_snlp_train_song_words)==1041496)

Could not load ../../../..//data/ohhla/train/www.ohhla.com/anonymous/nas/distant/tribal.nas.txt.html


Due to file encoding issues this code produces one error `Could not load ...`. **Ignore this error**.

## <font color='blue'>Task 1</font>: Develop and Train the Model

This is the core part of the assignment. You are to code up, train and tune a language model. Your language model needs to be subclass of the `lm.LanguageModel` class. You can use some of the existing language models developed in the lecture, or develop your own extensions. 

Concretely, you need to return a better language model in the `create_lm` function. This function receives a target vocabulary `vocab`, and it needs to return a language model defined over this vocabulary. 

The target vocab will be the union of the training and test set (hidden to you at development time). This vocab will contain words not in the training set. One way to address this issue is to use the `lm.OOVAwareLM` class discussed in the lecture notes.

In [10]:
import collections

In [11]:
class InterpolatedLM(lm.LanguageModel):
    def __init__(self, main, backoff, alpha):
        super().__init__(main.vocab, main.order)
        self.main = main
        self.backoff = backoff
        self.alpha = alpha
    def probability(self, word, *history):
        return self.alpha * self.main.probability(word,*history) + \
               (1.0 - self.alpha) * self.backoff.probability(word,*history)



In [12]:
class JelinekMercerLM(lm.LanguageModel):
    def __init__(self, unigram, alpha1, bigram, alpha2, trigram, alpha3, fourgram):
        super().__init__(unigram.vocab, fourgram.order)
        self.unigram = unigram
        self.bigram = bigram
        self.trigram  = trigram
        self.fourgram = fourgram
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.alpha3 = alpha3
        self.alpha4 = 1 - (alpha1 + alpha2 + alpha3)

    def probability(self, word, *history):

        return self.alpha1 * self.unigram.probability(word, *history) + \
               self.alpha2 * self.bigram.probability(word, * history) + \
               self.alpha3 * self.trigram.probability(word, *history) + \
               self.alpha4 * self.fourgram.probability(word, *history) 

In [13]:
class new_NGramLM(lm.CountLM):
    def __init__(self, train, order):
        """
        Create an NGram language model.
        Args:
            train: list of training tokens.
            order: order of the LM.
        """
        super().__init__(set(train), order)
        self._counts = collections.defaultdict(float)
        self._norm = collections.defaultdict(float)
        self._counts_history = collections.defaultdict(float)
        self._counts_word = collections.defaultdict(float)
        for i in range(self.order, len(train)):
            history = tuple(train[i - self.order + 1 : i])
            word = train[i]
            self._counts[(word,) + history] += 1.0
            self._norm[history] += 1.0
        for j in list(self._counts.keys()):
            
            self._counts_word[list(j)[0]] += 1.0
        for k in list(self._counts.keys()):
            self._counts_history[list(k)[1]] += 1.0
    def counts(self, word_and_history):
        #print(self._counts)
        #print(word_and_history, self._counts[word_and_history])
        return self._counts[word_and_history]
    def norm(self, history):
        #print(history, self._norm[history])
        return self._norm[history]
    def counts_history(self, history):
        return self._counts_history[history]
    def counts_word(self, word):
        return self._counts_word[word]
    def counts_all(self):
        return len(list(self._counts.keys()))

In [14]:
class KneseNeyLM(lm.LanguageModel):
    def __init__(self, main, backoff, discount):
        super().__init__(main.vocab, main.order)
        self.main = main
        self.backoff = backoff
        self.discount = discount
    def probability(self, word, *history):
        sub_history = tuple(history[-1:])
        word_and_history = (word,) + sub_history
        #print(word_and_history)
        history = "".join(sub_history)
        
        #if self.main.counts_history(history) == 0.0:
        if self.backoff.counts((history,)) == 0:
            
            return self.backoff.probability(word)

        else:
            
            p1 = max((self.main.counts(word_and_history) - self.discount),0)/ self.backoff.counts((history,))
            lmb = self.discount / self.backoff.counts((history,)) * self.main.counts_history(history)
            pc = self.main.counts_word(word) / self.main.counts_all()

            #print(p1+lmb*pc)

            return p1 + lmb * pc

In [15]:
oov_train = lm.inject_OOVs(_snlp_train_song_words)
bigram = lm.NGramLM(oov_train, 2)
unigram = lm.NGramLM(oov_train,1)
trigram = lm.NGramLM(oov_train,3)
fourgram = lm.NGramLM(oov_train,4)
#my_LM = JelinekMercerLM(unigram, 0.245, bigram, 0.505, trigram, 0.15, fourgram)
my_LM = KneseNeyLM(new_NGramLM(oov_train,2),unigram,0.75)
oov_vocab = set(oov_train)

In [16]:
## You should improve this cell
def create_lm(vocab):
    """
    Return an instance of `lm.LanguageModel` defined over the given vocabulary.
    Args:
        vocab: the vocabulary the LM should be defined over. It is the union of the training and test words.
    Returns:
        a language model, instance of `lm.LanguageModel`.
    """
    
    
    return lm.OOVAwareLM(my_LM, vocab - oov_vocab)

## <font color='green'>Setup 3</font>: Specify Test Data
This cell defines the directory to load the test songs from. Currently, this points to the dev set but when we evaluate your notebook we will point this directory elsewhere and use a **hidden test set**.  

In [17]:
#! SETUP 3
_snlp_test_dir = _snlp_book_dir + "/data/ohhla/dev"

## <font color='green'>Setup 4</font>: Load Test Data and Prepare Language Model
In this section we load the test data, prepare the reference vocabulary and then create your language model based on this vocabulary.

In [18]:
#! SETUP 4
_snlp_test_song_words = ohhla.words(ohhla.load_all_songs(_snlp_test_dir))
_snlp_test_vocab = set(_snlp_test_song_words)
_snlp_dev_vocab = set(_snlp_dev_song_words)
_snlp_train_vocab = set(_snlp_train_song_words)
_snlp_vocab = _snlp_test_vocab | _snlp_train_vocab | _snlp_dev_vocab
_snlp_lm = create_lm(_snlp_vocab)

## <font color='red'>Assessment 1</font>: Test Normalization (20 pts)
Here we test whether the conditional distributions of your language model are properly normalized. If probabilities sum up to $1$ you get full points, you get half of the points if probabilities sum up to be smaller than 1, and 0 points otherwise. Due to floating point issues we will test with respect to a tolerance $\epsilon$ (`_eps`).

Points:
* 10 pts: $\leq 1 + \epsilon$
* 20 pts: $\approx 1$

In [19]:
#! ASSESSMENT 1
_snlp_test_token_indices = [100, 1000, 10000]
_eps = 0.000001
for i in _snlp_test_token_indices:
    result = sum([_snlp_lm.probability(word, *_snlp_test_song_words[i-_snlp_lm.order+1:i]) for word in _snlp_vocab])
    print("Sum: {sum}, ~1: {approx_1}, <=1: {leq_1}".format(sum=result, 
                                                            approx_1=abs(result - 1.0) < _eps, 
                                                            leq_1=result - _eps <= 1.0))

Sum: 0.9999906122430835, ~1: False, <=1: True
Sum: 0.9845959458236123, ~1: False, <=1: True
Sum: 0.9433989656722234, ~1: False, <=1: True


The above solution is marked with **
<!-- ASSESSMENT 2: START_POINTS -->
20
<!-- ASSESSMENT 2: END_POINTS --> 
points **.

### <font color='red'>Assessment 2</font>: Apply to Test Data (50 pts)

We assess how well your LM performs on some unseen test set. Perplexities are mapped to points as follows.

* 0-10 pts: uniform perplexity > perplexity > 550, linear
* 10-30 pts: 550 > perplexity > 140, linear
* 30-50 pts: 140 > perplexity > 105, linear

The **linear** mapping maps any perplexity value between the lower and upper bound linearly to a score. For example, if uniform perplexity is $U$ and your model's perplexity is $P\leq550$, then your score is $10\frac{P-U}{550-U}$. 

In [20]:
lm.perplexity(_snlp_lm, _snlp_test_song_words)

166.12138115169657

The above solution is marked with **
<!-- ASSESSMENT 3: START_POINTS -->
0
<!-- ASSESSMENT 3: END_POINTS --> points**. 

## <font color='blue'>Task 2</font>: Describe your Approach

< Enter a 500 words max description of your model and the way you trained and tuned it here >

## <font color='red'>Assessment 3</font>: Assess Description (30 pts) 

We will mark the description along the following dimensions: 

* Clarity (10pts: very clear, 0pts: we can't figure out what you did)
* Creativity (10pts: we could not have come up with this, 0pts: Use the unigram model from the lecture notes)
* Substance (10pts: implemented complex state-of-the-art LM, 0pts: Use the unigram model from the lecture notes)

The above solution is marked with **
<!-- ASSESSMENT 1: START_POINTS -->
0
<!-- ASSESSMENT 1: END_POINTS --> points**. 